<a href="https://colab.research.google.com/github/SovetovAleksey/ReqSys/blob/4_quest/4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 55.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import implicit

import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.9/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3
users = np.arange(20)

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks][data['user_id'].isin(users)]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks][data['user_id'].isin(users)]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
1058,14,27021203242,4,834117,1,3.99,319,-1.00,1704,1,0.0,0.0
1059,14,27021203242,4,835788,1,1.00,319,-0.49,1704,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(3)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [ ]:
rec = MainRecommender(data_train)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
%%time
    
result['sim_items_rec'] = result['user_id'].apply(lambda x: rec.get_similar_items_recommendation(x))

sim_items_precision_at_k = result.apply(lambda row: precision_at_k(row['sim_items_rec'], row['actual']), axis=1).mean()
sim_items_recall_at_k = result.apply(lambda row: recall_at_k(row['sim_items_rec'], row['actual']), axis=1).mean()

print(f'similar_items_recommendation:  precision_at_k={sim_items_precision_at_k}, recall_at_k={sim_items_recall_at_k}')

In [ ]:
%%time
    
result['sim_users_rec'] = result['user_id'].apply(lambda x: rec.get_similar_users_recommendation(x))

sim_users_precision_at_k = result.apply(lambda row: precision_at_k(row['sim_users_rec'], row['actual']), axis=1).mean()
sim_users_recall_at_k = result.apply(lambda row: recall_at_k(row['sim_users_rec'], row['actual']), axis=1).mean()

print(f'similar_users_recommendation:  precision_at_k={sim_users_precision_at_k}, recall_at_k={sim_users_recall_at_k}')

In [ ]:
result.head(3)